<a href="https://colab.research.google.com/github/codyub/ESAA/blob/main/%ED%95%B8%EC%A6%88%EC%98%A8_7%EC%9E%A5_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8_p_245_271.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 앙상블 학습과 랜덤 포레스트
- 결정트리 앙상블: 랜덤포레스트
- 앙상블은 여러 괜찮은 예측기를 연결하여 더 좋은 예측기를 만든다. 
- 앙상블에는 배깅, 부스팅, 스태킹 등의 방법이 있음

# 7.1 투표 기반 분류기
- 직접 투표 분류기: 각 분류기의 예측을 모아 가장 많이 선택된 클래스로 예측
- 모든 분류기가 완벽하게 독립적이고 오차에 상관관계가 없어야 수학적 가설을 만족시킴
- 같은 데이터로 훈련하면 위 수학적 가설을 만족시키지 못함 -> 분류기들이 같은 종류의 오차를 만들기 쉬워 잘못된 클래스가 다수인 경우가 많아 앙상블 정확도가 낮아짐
- 다양한 분류기를 얻는 한 가지 방법은 각기 다른 알고리즘으로 학습시키는 것 -> 매우 다른 종류의 오차를 만들 가능성이 높아 앙상블 모델의 정확도가 향상 

In [1]:
# 데이터 로드
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y=make_moons(n_samples=100, noise=0.15)

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

# 투표 기반 분류기 만들어 훈련

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

voting_clf=VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)
     

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

#각 분류기의 테스트셋 정확도를 확인

In [3]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.9
SVC 0.9
VotingClassifier 0.9


어떻게 이렇게 성능이 다 똑같지?

# 7.2 배깅과 페이스팅

같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성해 분류기를 각각 다르게 학습하는 것.
- 배깅: 훈련 세트에서 중복을 허용하여 샘플링 하는 방식을 배깅이라고 함
- 페이스팅: 중복을 허용하지 않음
- 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아 새로운 샘ㅍ르에 대한 예측을 만든다.
- 수집 함수는 전형적으로 분류일때는 통계적 최빈값, 회귀에 대해서는 평균을 계산
- 개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 크게 편향되어 있지만, 수집 함수를 통과하면 편향과 분산이 모두 감소함 
- 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만, 분산은 줄어든다. 

# 7.2.1 사이킷런의 배깅과 페이스팅
- 결정 트리 분류기 500개의 앙상블 훈련시키는 코드
- 각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련(배깅의 경우)

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred=bag_clf.predict(X_test)

- 부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로, 배깅이 페이스팅보다 편향이 조금 더 높음
- 다양성 추가 = 예측기들의 상관관계를 줄임 = 앙상블의 분산을 감소

# 7.2.2 oob 평가
- 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링 되고, 어떤 것은 전혀 선택되지 않을 수 있음 
- BaggingClassifier는 기본값으로 중복을 허용해 훈련 세트의 크기 만큼의 m개의 샘플을 선택한다. = 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링 됨
- 나머지 37% 새믈을 out of bag 샘플이라고 한다. -> 예측기마다 남겨진 37%는 다르다
- oob score=true로 하면, 훈련이 끝난 후 자동으로 oob 평가를 수행함. 

In [5]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.95

In [6]:
from sklearn.metrics import accuracy_score
y_pred=bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.85

In [7]:
bag_clf.oob_decision_function_ # [음성클래스 속할 확률, 양성 클래스 속할 확률]

array([[0.9673913 , 0.0326087 ],
       [0.05025126, 0.94974874],
       [0.99456522, 0.00543478],
       [0.59685864, 0.40314136],
       [0.02777778, 0.97222222],
       [0.03977273, 0.96022727],
       [0.01734104, 0.98265896],
       [1.        , 0.        ],
       [0.30601093, 0.69398907],
       [1.        , 0.        ],
       [0.04371585, 0.95628415],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.95049505, 0.04950495],
       [0.03428571, 0.96571429],
       [0.1576087 , 0.8423913 ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.58333333, 0.41666667],
       [0.33888889, 0.66111111],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.31666667, 0.68333333],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.88421053, 0.11578947],
       [0.99453552, 0.00546448],
       [0.08108108, 0.91891892],
       [0.

# 7.3 랜덤 패치와 랜덤 서브스페이스
- BaggingClassifier는 특정 샘플링도 지원함
- max_samples, bootstrap과 동일하지만, 샘플이 아니고 특성에 대한 샘플링이라 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련된다.

- 이미지 같은 매우 고차원의 데이터셋을 다룰 때 유용하다. 훈련 특성과 샘플을 모두 샘플링하는 것: 랜덤 패치 방식
- 훈련 샘플을 모두 사용하고, 특성은 샘플링하는 것: 랜덤 서브스페이스 방식

- 특성 샘플링은 편향은 늘리지만 분산은 낮춘다.

# 7.4 랜덤포레스트
- 랜덤 포레스트는 일반적으로 배깅 방법을 적용한 결정 트리의 앙상블
- 다음은 최대 16개의 리프 노드를 갖는 500개 트리로 이뤄진 랜덤 포레스트 분류기를 여러 CPU 코어에서 훈련시키느느 훈련

In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf=RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf=rnd_clf.predict(X_test)

- 랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 주입
- 이는 결국 트리를 더욱 다양하게 만들고, 편향을 손해보는 대신, 분산을 낮춰 전체적으로 더 훌륭한 모델을 만든다. 

In [9]:
bag_clf=BaggingClassifier(
    DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16), n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

# 7.4.1 엑스트라 트리
- 랜덤 포레스트에서 트리를 만들 때, 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용
- 트리를 더 무작위로 만들기 위해 최적의 임곗값을 찾는 대신, 후보 특성을 사용해 무작위로 분할한 다음 그중에서 최상의 분할을 선택
- 위를 익스트림 랜덤 트리라고 칭함
- 편향이 늘어나지만, 분산을 낮춤.
- 모든 노드에서 특성마다 가장 최적의 임곗값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요되는 작업 중 하나이므로, 일반적인 랜덤 포레스트 보다 엑스트라 트리가 훨씬 빠르다. 

# 7.4.2 특성 중요도
- 랜덤 포레스트의 또다른 장점: 특성의 상대적 중요도를 측정하기 쉽다
- 사이킷런은 어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인해 특성의 중요도를 측정
- 가중치 평균, 각 노드의 가중치는 연관된 훈련 샘플이 수와 같음. 

In [10]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.08708165433265928
sepal width (cm) 0.024556361703660778
petal length (cm) 0.43187450577169356
petal width (cm) 0.4564874781919862


- 랜덤 포레스트는 특히 특성을 선택해야 할 때, 어떤 특성이 중요한지 빠르게 확인할 수 있어 매우 편리

# 7.5 부스팅
- 부스팅은 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법
- 앞의 모델의 보완해 나가면서 일련의 예측기를 학습시키는 직렬 방식
- 에이다 부스트: 가장 인기
- 그레이디언트 부스팅

# 7.5.1 에이다 부스트
- 이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것
- 이렇게 하면, 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞추게 된다. 
- ex. 에이다 부스트 분류기를 만들 때, 먼저 알고리즘이 기반이 되는 첫 번째 분류기를 훈련 세트에서 훈련시키고 예측을 만든다. 그 다음 알고리즘이 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높임 
- 두 번째 분류기는 업데이트 된 가중치를 사용해 훈련 세트엣 훈련하고 다시 예측을 만든다. 그 다음 다시 가중치를 업데이트 하는 식으로 계속된다. 

- 200개의 아주 얕은 결정 트리를 기반으로 하는 에이다 부스트 분류기를 훈련시키기

In [11]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

# 7.5.2 그레이디언트 부스팅
- 앙상블에 이전까지 오차를 보정하도록 예측기를 순차적으로 추가
- 하지만, 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신, 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킴


- 결정 트리를 기반 예측기로 하는 회귀 문제 풀이: 그레이디언트 트리 부스팅, 그레이디언트 부스티드 회귀 트리

In [12]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

첫 번째 예측기에서 생긴 잔여 오차에 두 번째 descisiontreeregressor를 훈련

In [13]:
y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

두 번째 예측기에서 생긴 잔여 오차에 세 번째 회귀 모델을 훈련

In [14]:
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

두 번째 예측기에서 생긴 잔여 오차에 세 번째 회귀 모델을 훈련

In [22]:
# 'X_new'가 없어서 실행하지 않음. 
# y_pred=sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

- 새로운 앙상블 모델이 생김
- 새로운 샘플에 대한 예측을 만들기 위해서는 모든 트리의 예측을 더하면 됨

# 사이킷런의 GradientBoostingRegressor 사용해 GBRT 앙상블 훈련
- 매개변수 max_depth, min_samples_leaf

In [17]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

- learning_rate 매개변수 : 각 트리의 기여 정도 조절

- 규제방법 축소: 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리 필요하지만 일반적으로 예측 성능은 좋아짐
- 최적의 트리 수를 찾기 위해서는 조기 종료 기법 사용 -> staged_predict() 메서드

- 훈련의 각 단계에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자 반환

# 120개의 트리로 GBRT 앙상블 훈련
- 최적의 트리 수 찾기 위해 각 훈련 단게에서 검증 오차 측정 
- 새로운 GBRT 앙상블 훈련

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val=train_test_split(X, y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors=[mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=119)

- 실제로 훈련 중지하는 방법으로 조기 종료 구현: warm_start = True로 설정
-> 사이킷런이 fit() 메서드가 호출될 때 기존 트리 유지하고 훈련 추가할 수 있도록 해줌

# 연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련 멈추기

In [23]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up += 1
    if error_going_up == 5:
      break    # 조기 종료

- 각 트리가 훈련할 때 사용할 훈련 샘플의 비율 지정할 수 있는 subsample 매개변수 지원

- 확률적 그레이디언트 부스팅: 편향이 높아지는 대신 분산 낮아져. 훈련 속도 상당히 높임
- 최적화된 그레이디언트 부스팅 구현으로 XGBoost 파이썬 라이브러리

In [20]:
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred=xgb_reg.predict(X_val)

# 자동 조기 종료와 같은 여러 좋은 기능도 제공

In [21]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.41021
[1]	validation_0-rmse:0.36178
[2]	validation_0-rmse:0.33938
[3]	validation_0-rmse:0.33127
[4]	validation_0-rmse:0.33005
[5]	validation_0-rmse:0.33105
[6]	validation_0-rmse:0.33333


/usr/local/lib/python3.9/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


# 7.6 스태킹
스태킹 : '앙상블에 속한 모든 예측기의 예측을 수합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까요?'라는 기본 아이디어로 출발

- 각각 다른 값을 예측하고 마지막 예측기(블렌더 또는 메타 학습기)가 예측을 입력으로 받아 최종 예측.

- 블렌더를 학습시키는 일반적인 방법: 홀드 아웃 세트 이용

1. 훈련 세트를 두 개의 서브셋으로 나눈다 -> 첫 번째 서브셋은 첫 번째 레이어의 예측을 훈련시키기 위해 사용. 
2. 첫 번째 레이어의 예측기를 사용해 두 번째 세트에 대한 예측 만들어. -> 타깃값을 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만들 수 있다. 
3. 블렌더가 새 훈련 세트로 훈련돼. 즉, 첫 번째 레이어의 예측을 가지고 타깃값을 예측하도록 학습된다

- 이런 방식의 블렌더를 여러 개 훈련시키는 것도 가능. 
-> 블렌더만의 레이어 
- 훈련 세트를 세 개의 서브셋으로 나눈다 -> 첫 번째 세트는 첫 번째 레이어를 훈련시키는 데 사용되고 나머지 세트도 각 레이어를 훈련시키는 데 사용돼 -> 각 레이어를 차례대로 실행해서 새로운 샘플에 대한 예측 만들 수 있다

- 사이킷런은 스태킹을 직접 지원하지 않는다